In [29]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="af3a4e21d9974f798b0ddef081728f2b",  client_secret="99a65d20eff04d64bcf24b11824dffc4"))



Get songs from a playlist 

In [ ]:
results = sp.user_playlist_tracks("spotify", "5SVTjzAeA1MwGGbKJ4pBHU")
tracks = results['items']

for oset in range(100,results['total'],100):
    results = sp.user_playlist_tracks("spotify", "5SVTjzAeA1MwGGbKJ4pBHU", offset=oset)
    tracks += results['items']
len(tracks)
results

In [ ]:
import pprint
pprint.pprint(results) # nested view is clearer and easier to parse by eye

The problem we have is that the data are in JSON - supposedly 'human readable' byt not what we need. Therefore, we need to parse the JSON and extract the info we want, namely:
- Artist name
- Track name
- Album name
- Track ID (this'll be useful later when we join some other data and will act as our primary key)

<style>
r { color: Red }
o { color: Orange }
g { color: Green }
</style>

In this example, to get the artist name, we need to go to the 'key' track, then down to 'artists', then down to 'name'

```{python}
artist_name = track['track']['artists'][0]['name'] # gets the first artist (0)
```


<r>'track':</r> {'album': {'album_type': 'album',

'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/04aQfpx1U1WQEJkrvGs4ig'},
'href': 'https://api.spotify.com/v1/artists/04aQfpx1U1WQEJkrvGs4ig',

'id': '04aQfpx1U1WQEJkrvGs4ig',

<r>'name':</r> 'Dalida',

<r>'type':</r> 'artist',

'uri': 'spotify:artist:04aQfpx1U1WQEJkrvGs4ig'}],
                                             






In [68]:
tracks = results['items']

# Empty list to store dictionaries
rows = []

# Iterate over each track in the 'items' list
for track in tracks:
    # Extract the artist name, track name, album name, and track ID from the JSON
    artist_name = track['track']['artists'][0]['name'] # gets the first artist (0)
    track_name = track['track']['name']
    album_name = track['track']['album']['name']
    track_id = track['track']['id']
    
    # Create a dictionary with the extracted values
    track_dict = {'Artist': artist_name, 'Track': track_name, 'Album': album_name, 'Track ID': track_id}
    
    # Append the dictionary to the list
    rows.append(track_dict)

# Create a dataframe from the list of dictionaries
playistFrance = pd.DataFrame(rows)

# Display the resulting dataframe
playistFrance.head()

,Artist,Track,Album,Track ID
0,Dalida,Laissez-moi danser,LES NUMEROS UN DE DALIDA,08ntOTgpfOo1ZCqUgsxeGo
1,Au P'Tit Bonheur,J'veux du soleil,Le Mal De Vivre,0L4JEAhghqIcwjVyMJv8Yw
2,Images,Les démons de minuit,Best Of Images & Début De Soirée,3utIJ1ebqgyof7OyPLaPAg
3,Philippe Cataldo,Les divas du dancing,Les divas du dancing,5oKu1Ey6XNDXv5B3LG7kn9
4,Michel Sardou,Être une femme,L'album de sa vie 100 titres,1c3Covtj7j3rEisBGmhFNO


Get the audio features

In [71]:
# Retrieve the tracks from the playlist
results = sp.user_playlist_tracks("spotify", "5SVTjzAeA1MwGGbKJ4pBHU")
tracks = results['items']

# Extract the track IDs from the tracks
track_ids = [track['track']['id'] for track in tracks]

In [73]:
audio_features = sp.audio_features(track_ids[:100]) ## [get the audio features for the tracks with the ids we extracted from the playllist

audioFeatures = pd.DataFrame(audio_features) #Now turn them into vectors and a dataframe

audioFeatures.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.664,0.968,11,-3.775,0,0.1770,0.29100,0.000000,0.2410,0.639,134.116,audio_features,08ntOTgpfOo1ZCqUgsxeGo,spotify:track:08ntOTgpfOo1ZCqUgsxeGo,https://api.spotify.com/v1/tracks/08ntOTgpfOo1...,https://api.spotify.com/v1/audio-analysis/08nt...,225840,4
1,0.610,0.634,11,-13.052,0,0.1540,0.24500,0.000000,0.2060,0.876,111.364,audio_features,0L4JEAhghqIcwjVyMJv8Yw,spotify:track:0L4JEAhghqIcwjVyMJv8Yw,https://api.spotify.com/v1/tracks/0L4JEAhghqIc...,https://api.spotify.com/v1/audio-analysis/0L4J...,182800,4
2,0.682,0.818,4,-6.129,0,0.0300,0.00434,0.000005,0.0992,0.894,118.926,audio_features,3utIJ1ebqgyof7OyPLaPAg,spotify:track:3utIJ1ebqgyof7OyPLaPAg,https://api.spotify.com/v1/tracks/3utIJ1ebqgyo...,https://api.spotify.com/v1/audio-analysis/3utI...,233640,4
3,0.711,0.868,1,-7.146,1,0.0495,0.08500,0.000001,0.0637,0.735,102.916,audio_features,5oKu1Ey6XNDXv5B3LG7kn9,spotify:track:5oKu1Ey6XNDXv5B3LG7kn9,https://api.spotify.com/v1/tracks/5oKu1Ey6XNDX...,https://api.spotify.com/v1/audio-analysis/5oKu...,242414,4
4,0.612,0.915,5,-5.365,1,0.1250,0.25300,0.000000,0.4250,0.738,129.243,audio_features,1c3Covtj7j3rEisBGmhFNO,spotify:track:1c3Covtj7j3rEisBGmhFNO,https://api.spotify.com/v1/tracks/1c3Covtj7j3r...,https://api.spotify.com/v1/audio-analysis/1c3C...,252853,4


In [74]:
songsFrance = pd.merge(audioFeatures, playistFrance, left_on='id', right_on='Track ID') # join on id and track id

# Print the merged DataFrame
songsFrance.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,Artist,Track,Album,Track ID
0,0.664,0.968,11,-3.775,0,0.1770,0.29100,0.000000,0.2410,0.639,...,08ntOTgpfOo1ZCqUgsxeGo,spotify:track:08ntOTgpfOo1ZCqUgsxeGo,https://api.spotify.com/v1/tracks/08ntOTgpfOo1...,https://api.spotify.com/v1/audio-analysis/08nt...,225840,4,Dalida,Laissez-moi danser,LES NUMEROS UN DE DALIDA,08ntOTgpfOo1ZCqUgsxeGo
1,0.610,0.634,11,-13.052,0,0.1540,0.24500,0.000000,0.2060,0.876,...,0L4JEAhghqIcwjVyMJv8Yw,spotify:track:0L4JEAhghqIcwjVyMJv8Yw,https://api.spotify.com/v1/tracks/0L4JEAhghqIc...,https://api.spotify.com/v1/audio-analysis/0L4J...,182800,4,Au P'Tit Bonheur,J'veux du soleil,Le Mal De Vivre,0L4JEAhghqIcwjVyMJv8Yw
2,0.682,0.818,4,-6.129,0,0.0300,0.00434,0.000005,0.0992,0.894,...,3utIJ1ebqgyof7OyPLaPAg,spotify:track:3utIJ1ebqgyof7OyPLaPAg,https://api.spotify.com/v1/tracks/3utIJ1ebqgyo...,https://api.spotify.com/v1/audio-analysis/3utI...,233640,4,Images,Les démons de minuit,Best Of Images & Début De Soirée,3utIJ1ebqgyof7OyPLaPAg
3,0.711,0.868,1,-7.146,1,0.0495,0.08500,0.000001,0.0637,0.735,...,5oKu1Ey6XNDXv5B3LG7kn9,spotify:track:5oKu1Ey6XNDXv5B3LG7kn9,https://api.spotify.com/v1/tracks/5oKu1Ey6XNDX...,https://api.spotify.com/v1/audio-analysis/5oKu...,242414,4,Philippe Cataldo,Les divas du dancing,Les divas du dancing,5oKu1Ey6XNDXv5B3LG7kn9
4,0.612,0.915,5,-5.365,1,0.1250,0.25300,0.000000,0.4250,0.738,...,1c3Covtj7j3rEisBGmhFNO,spotify:track:1c3Covtj7j3rEisBGmhFNO,https://api.spotify.com/v1/tracks/1c3Covtj7j3r...,https://api.spotify.com/v1/audio-analysis/1c3C...,252853,4,Michel Sardou,Être une femme,L'album de sa vie 100 titres,1c3Covtj7j3rEisBGmhFNO


Only 86 songs but it's a start

In [75]:
songsFrance.shape

(86, 22)

# SCRATCH PAPER here be 🐉🐉🐉

In [ ]:
def get_playlist_tracks(username, playlist_id):
    
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    return tracks

tracks = get_playlist_tracks("spotify", "12Wbv8sIx84T5uh6iOoJ7V")
tracks

In [48]:
# Getting all the artists

def get_artists_from_playlist(playlist_id):
    
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    
    artists = []
    
    for track in tracks_from_playlist:
        artists_info = track['track']['artists']
        
        for artist_info in artists_info:
            artists.append(artist_info['name'])
    
    return list(set(artists))

In [49]:
# Getting all the artists IDs

def get_artists_ids_from_playlist(playlist_id):
    
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    
    artists_ids = []
    
    for track in tracks_from_playlist:
        artists_info = track['track']['artists']
        
        for artist_info in artists_info:
            artists_ids.append(artist_info['id'])
            
    return list(set(artists_ids))


In [ ]:
artists_ids = get_artists_ids_from_playlist("12Wbv8sIx84T5uh6iOoJ7V")
artists_ids